# Load and test a pre-trained model

In [7]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os
import sys
import cloudpickle
import pickle
import torch
if not hasattr(cloudpickle.cloudpickle, '_make_function') and hasattr(cloudpickle.cloudpickle_fast, '_make_function'):
    print("Applying cloudpickle monkey patch")
    cloudpickle.cloudpickle._make_function = cloudpickle.cloudpickle_fast._make_function

from gym_utils import SMBRamWrapper, load_smb_env, SMB
version = 'SuperMarioBros-1-1-v0'

In [ ]:
MODEL_DIR = './models/v3'

crop_dim = [0, 16, 0, 13]
n_stack = 4
n_skip = 4
MODEL_NAME = 'best_model_9800000'

In [16]:
print("Loading environment...")
env_wrap = load_smb_env(version, crop_dim, n_stack, n_skip)
print("Environment loaded successfully")

# Load model with custom_objects (the method that worked)
print(f"Loading model from: {os.path.join(MODEL_DIR, MODEL_NAME)}")
custom_objects = {
    "learning_rate": 0.0,
    "lr_schedule": lambda _: 0.0,
    "clip_range": lambda _: 0.0,
}
model = PPO.load(os.path.join(MODEL_DIR, MODEL_NAME), env=env_wrap, custom_objects=custom_objects)
print("Model loaded successfully")

# Create SMB instance
smb = SMB(env_wrap, model)
print("Successfully created SMB instance")

Loading environment...
Environment loaded successfully
Loading model from: ./models\pre-trained-1
Model loaded successfully
Successfully created SMB instance


In [17]:
try:
    print("Starting game play...")
    rewards, steps = smb.play(episodes=1, deterministic=True, render=True, return_eval=True)
    print(f"Episode completed! Average reward: {sum(rewards)/len(rewards):.2f}, Steps: {sum(steps)}")
except Exception as e:
    print(f"Game play error: {e}")
finally:
    # Clean up resources properly to avoid window crashes
    print("Cleaning up resources...")
    try:
        env_wrap.close()
        print("Environment closed successfully")
    except Exception as close_error:
        print(f"Error closing environment: {close_error}")
    print("Game session ended")

Starting game play...
Episode:1 Score:[3066.]
Game play error: unsupported operand type(s) for +: 'int' and 'dict'
Cleaning up resources...
Environment closed successfully
Game session ended
